In [124]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
from scipy.stats import beta, gamma, kstest, ks_2samp, mannwhitneyu

skip = 5
percentile_to_skip = 0.97
size_of_bins_durations = 2
size_of_bins_dynamics = 0.1
statistical_significance = 0.05

# Ignorowanie ostrzeżeń
warnings.filterwarnings('ignore')

In [118]:
data = pd.read_csv('us500.csv')

data.drop(['Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)

data['Local time'] = data['Local time'].apply(lambda row: row[:-13])

data['Local time'] = pd.to_datetime(data['Local time'],  format="%d.%m.%Y %H:%M:%S")

filtered_df = data[(data['Local time'].dt.time >= pd.to_datetime('15:30:00').time()) & (data['Local time'].dt.time <= pd.to_datetime('22:00:00').time())] 

filtered_df.reset_index(inplace=True, drop=True)

ema_12 = filtered_df['Close'].ewm(span=12, adjust=False).mean()
ema_26 = filtered_df['Close'].ewm(span=26, adjust=False).mean()
histogram = (ema_12 - ema_26) - (ema_12 - ema_26).ewm(span=9, adjust=False).mean()

# Wyznaczenie sygnału wzrostowego/spadkowego (jest zmieniane w trakcie obliczeń aby zapewnić funkcjonaność)
sign = np.sign(histogram)

# Kopia zapisująca przefiltrowany sygnał
new_sign = np.zeros(len(sign))

# Filtrowanie 
for i in range(1, len(sign)):
            
    # Przypadek gdy nie ma jeszcze określonego trendu
    if i == 0 or np.all(new_sign[0:i] == 0):

        # Nie da się dalej określić trendu
        if np.sum(sign[i:i+skip]) != skip and np.sum(sign[i:i+skip]) != -skip:
            continue
                    
        # Da się określić trend
        else:
            new_sign[i] = sign[i]
            
    # Zmiana sygnału (sign[i-1] jest już przefiltrowany więc nie odnosi się do początkowej wartości)
    elif sign[i] != sign[i-1]:

        # Zmiana znaku, jeżeli nie ma spełnionego warunku
        if np.sum(sign[i:i+skip]) != skip and np.sum(sign[i:i+skip]) != -skip:
            sign[i] = sign[i-1]
            new_sign[i] = sign[i]

        else:
            new_sign[i] = sign[i]

    else:
        new_sign[i] = sign[i]

last_change = 0

periods = []

# Wyznaczenie okresów wzrostowych/spadkowych
for i in range(1, len(new_sign)):
            
    if new_sign[i] != new_sign[i-1] or i==len(new_sign)-1:
                    
        # Warunek wyrzuca pierwszy przypadek jezeli ma mniej niz 5 dni (tylko on może mieć mniej niż 5 dni, reszta będzie miała więcej niż 5)
        if i - last_change >= 5:
                            
            periods.append((last_change, i, new_sign[i-1] > 0))

        last_change = i

# Punkty ekstremalne - 'True' dla punktu maksymalnego oraz 'False' dla punktu minimalnego
filtered_df['Extremes'] = None

values = filtered_df['Close'].tolist()

# Wyznaczenie maksimów i minimów wśród okresów spadkowych
for start, end, is_max in periods:
            
    # Maksymalny element
    if is_max:
        max_el = max(values[start:end])
        filtered_df['Extremes'][(start + values[start:end].index(max_el))] = True # indeks + indeks maksymalnego elementu w wycinku listy
            
    # Minimalny element
    elif not is_max:
        min_el = min(values[start:end])
        filtered_df['Extremes'][(start + values[start:end].index(min_el))] = False # indeks + indeks minimalnego elementu w wycinku listy

df = filtered_df

In [119]:
table_of_durations_ups = []
table_of_durations_downs = []

table_of_dynamics_ups = []
table_of_dynamics_downs = []

last_extremum_type = None
last_extremum_index = None

for i in range(0, len(df)):
    
    if df['Extremes'].iloc[i] is not None:
        if last_extremum_type is not None:
            if last_extremum_type == False:
                table_of_durations_ups.append(i - last_extremum_index)
                table_of_dynamics_ups.append(df['Close'].iloc[i] - df['Close'].iloc[last_extremum_index])
            else:
                table_of_durations_downs.append(i - last_extremum_index)
                table_of_dynamics_downs.append(df['Close'].iloc[i] - df['Close'].iloc[last_extremum_index])
        last_extremum_type = df['Extremes'].iloc[i]
        last_extremum_index = i


# Use the values from the first column
table_of_durations_ups = np.array(table_of_durations_ups)
table_of_durations_downs = np.array(table_of_durations_downs)

table_of_dynamics_ups = np.array(table_of_dynamics_ups)
table_of_dynamics_downs = np.abs(np.array(table_of_dynamics_downs))

In [120]:
def statiscits_for_data(table, percentile_to_skip, size_of_bins, statistical_significance, xaxis_title):

    data = table[table < np.quantile(table, percentile_to_skip)]

    fig = go.Figure(data=[go.Histogram(x=data, name='dane rzeczywiste', histnorm='probability', xbins=dict(
                                        start=0,  # Początkowy bin
                                        end=max(data),    # Końcowy bin
                                        size=size_of_bins           # Szerokość każdego bina
                                    ))])  # nbinsx określa liczbę słupków

    # Część teoretyczna wyznaczania rozkladów gamma oraz beta
    data_scaled = data / data.max()

    # Estymacja parametrów rozkładu
    a_est, b_est, _, _ = beta.fit(data_scaled)
    shape_est, loc_est, scale_est = gamma.fit(data_scaled) 

    # Test KS z użyciem estymowanych parametrów
    ks_stat, p_value = kstest(data_scaled, 'beta', args=(a_est, b_est))
    ks_stat_gamma, p_value_gamma = kstest(data_scaled, 'gamma', args=(shape_est, loc_est, scale_est))

    random_data = gamma.rvs(shape_est, loc= loc_est,scale=scale_est, size=10000)

    random_data = random_data * data.max()

    fig.add_trace(go.Histogram(x=random_data, histnorm='probability', name='rozkład gamma', xbins=dict(
                                        start=0,  # Początkowy bin
                                        end=max(random_data),    # Końcowy bin
                                        size=size_of_bins           # Szerokość każdego bina
                                    )))

    if p_value_gamma > statistical_significance:
        decision = 'powyżej'
    else:
        decision = 'poniżej'

    # Dodanie tytułu i etykiet osi
    fig.update_layout(title=f'Histogram - p-wartość {decision} poziomu istotności dla rozkładu gamma',
                    xaxis_title=xaxis_title,
                    yaxis_title='Liczba okresów')

    print(f"KS statistic beta: {ks_stat}, p-value: {p_value}")
    print(f"KS statistic gamma: {ks_stat_gamma}, p-value: {p_value_gamma}")

    print('Wartość oczekiwana: ',np.mean(data))
    print('Mediania: ', np.median(data))

    print('Wartość oczekiwana z rozkładu gamma: ',np.mean(random_data))
    print('Mediania z rozkładu gamma: ', np.median(random_data))

    percentils = [0.80, 0.90, 0.95, 0.99]

    # Obliczanie kwantyla
    quantile_values = gamma.ppf(percentils, a=shape_est, scale=scale_est, loc=loc_est) * data.max()
    quantile_values_real = np.quantile(data, percentils)

    print('Kwantyle dla wartości rzeczywistych (80%, 90%, 95%, 99%):',  quantile_values_real)
    print('Kwantyle dla rozkładu gamma (80%, 90%, 95%, 99%):',  quantile_values)

    i = 0

    for i in range(len(quantile_values)):
        # fig.add_vline(x=quantile_values[i], line_dash="dash", line_color="red", annotation_text=f'{percentils[i] * 100}% ({int(quantile_values[i])})', annotation_position = "top")
        # fig.add_vline(x=quantile_values_real[i], line_dash="dash", line_color="blue", annotation_text=f'{percentils[i] * 100}% ({int(quantile_values_real[i])})', annotation_position = "top")
        i+=1

    fig.show()

    statistic, p_value = ks_2samp(table_of_durations_ups, table_of_durations_downs)

    print(f"Statystyka KS: {statistic}")
    print(f"P-wartość: {p_value}")

    statistic, p_value = mannwhitneyu(table_of_durations_ups, table_of_durations_downs, alternative='two-sided', )

    print(f"Statystyka U: {statistic}")
    print(f"P-wartość: {p_value}")


    # Przykładowe dane
    data1 = table_of_durations_ups
    data2 = table_of_durations_downs

    # Liczba bootstrapowych replikacji
    n_bootstraps = 1000
    bootstrapped_means_diff = []

    # Pętla bootstrap
    for _ in range(n_bootstraps):
        sample1 = np.random.choice(data1, size=data1.size, replace=True)
        sample2 = np.random.choice(data2, size=data2.size, replace=True)
        diff = np.mean(sample1) - np.mean(sample2)
        bootstrapped_means_diff.append(diff)

    # Obliczenie statystyk
    bootstrapped_means_diff = np.array(bootstrapped_means_diff)
    ci_lower = np.percentile(bootstrapped_means_diff, 2.5)
    ci_upper = np.percentile(bootstrapped_means_diff, 97.5)
    p_value = np.mean(bootstrapped_means_diff >= 0 if np.mean(data1) < np.mean(data2) else bootstrapped_means_diff <= 0)

    print(f"95% przedział ufności dla różnicy średnich według bootsrapu: ({ci_lower}, {ci_upper})")
    print(f"P-wartość: {p_value}")

    # Przykładowe dane
    data1 = table_of_durations_ups
    data2 = table_of_durations_downs

    # Liczba bootstrapowych replikacji
    n_bootstraps = 1000
    bootstrapped_medians_diff = []

    # Pętla bootstrap
    for _ in range(n_bootstraps):
        sample1 = np.random.choice(data1, size=data1.size, replace=True)
        sample2 = np.random.choice(data2, size=data2.size, replace=True)
        diff = np.median(sample1) - np.median(sample2)
        bootstrapped_medians_diff.append(diff)

    # Obliczenie statystyk
    bootstrapped_medians_diff = np.array(bootstrapped_medians_diff)
    ci_lower = np.percentile(bootstrapped_medians_diff, 2.5)
    ci_upper = np.percentile(bootstrapped_medians_diff, 97.5)
    p_value = np.mean(bootstrapped_medians_diff >= 0 if np.median(data1) < np.median(data2) else bootstrapped_medians_diff <= 0)

    print(f"95% przedział ufności dla różnicy median według bootsrapu: ({ci_lower}, {ci_upper})")
    print(f"P-wartość: {p_value}")

In [125]:
print("Długość trwania wzrostów")
statiscits_for_data(table_of_durations_ups, percentile_to_skip, size_of_bins_durations, statistical_significance, xaxis_title='Czas trwania')

print("\n\nDługość trwania spadków")
statiscits_for_data(table_of_durations_downs, percentile_to_skip, size_of_bins_durations, statistical_significance, xaxis_title='Czas trwania')

print("\n\nDynamika wzrostów")
statiscits_for_data(table_of_dynamics_ups, percentile_to_skip, size_of_bins_dynamics, statistical_significance, xaxis_title='Wartość')

print("\n\nDynamika spadków")
statiscits_for_data(table_of_dynamics_downs, percentile_to_skip, size_of_bins_dynamics, statistical_significance, xaxis_title='Wartość')

Długość trwania wzrostów
KS statistic beta: 0.14308581128856057, p-value: 5.997216518972964e-08
KS statistic gamma: 0.06362682641037432, p-value: 0.06436814370870791
Wartość oczekiwana:  20.381861575179
Mediania:  19.0
Wartość oczekiwana z rozkładu gamma:  20.218632330715
Mediania z rozkładu gamma:  17.698375764074108
Kwantyle dla wartości rzeczywistych (80%, 90%, 95%, 99%): [29. 37. 42. 49.]
Kwantyle dla rozkładu gamma (80%, 90%, 95%, 99%): [29.10303809 36.49340428 43.4493581  58.66603333]


Statystyka KS: 0.15196141617255307
P-wartość: 7.525557912034457e-05
Statystyka U: 109249.0
P-wartość: 1.0163759592095675e-05
95% przedział ufności dla różnicy średnich według bootsrapu: (-13.35891364827705, 8.61618087780356)
P-wartość: 0.353
95% przedział ufności dla różnicy median według bootsrapu: (3.0, 7.0)
P-wartość: 0.0


Długość trwania spadków
KS statistic beta: 0.5881881566490188, p-value: 7.431719492691995e-138
KS statistic gamma: 0.05145451539764617, p-value: 0.21213129205510617
Wartość oczekiwana:  17.004796163069546
Mediania:  14.0
Wartość oczekiwana z rozkładu gamma:  17.068741626977204
Mediania z rozkładu gamma:  14.733150097041332
Kwantyle dla wartości rzeczywistych (80%, 90%, 95%, 99%): [25.   32.   39.2  48.84]
Kwantyle dla rozkładu gamma (80%, 90%, 95%, 99%): [24.94968341 31.89590622 38.47941546 52.98554293]


Statystyka KS: 0.15196141617255307
P-wartość: 7.525557912034457e-05
Statystyka U: 109249.0
P-wartość: 1.0163759592095675e-05
95% przedział ufności dla różnicy średnich według bootsrapu: (-13.200958150726134, 8.337093833247401)
P-wartość: 0.357
95% przedział ufności dla różnicy median według bootsrapu: (3.0, 7.0)
P-wartość: 0.0


Dynamika wzrostów
KS statistic beta: 1.0, p-value: 0.0
KS statistic gamma: 0.060172954654156174, p-value: 0.09235187335110517
Wartość oczekiwana:  1.2254415274463013
Mediania:  1.009999999999991
Wartość oczekiwana z rozkładu gamma:  1.2284208318307417
Mediania z rozkładu gamma:  1.069938799895976
Kwantyle dla wartości rzeczywistych (80%, 90%, 95%, 99%): [1.774  2.38   2.862  3.7864]
Kwantyle dla rozkładu gamma (80%, 90%, 95%, 99%): [1.78731083 2.27411394 2.73457019 3.74703605]


Statystyka KS: 0.15196141617255307
P-wartość: 7.525557912034457e-05
Statystyka U: 109249.0
P-wartość: 1.0163759592095675e-05
95% przedział ufności dla różnicy średnich według bootsrapu: (-14.218976647761453, 8.712484021869898)
P-wartość: 0.376
95% przedział ufności dla różnicy median według bootsrapu: (3.0, 7.0)
P-wartość: 0.0


Dynamika spadków
KS statistic beta: 0.9976076555023924, p-value: 0.0
KS statistic gamma: 0.04714023107758858, p-value: 0.30137145893825634
Wartość oczekiwana:  1.1725598086124407
Mediania:  0.9350000000000023
Wartość oczekiwana z rozkładu gamma:  1.1765757623618958
Mediania z rozkładu gamma:  1.0049725836024601
Kwantyle dla wartości rzeczywistych (80%, 90%, 95%, 99%): [1.73  2.27  2.996 3.77 ]
Kwantyle dla rozkładu gamma (80%, 90%, 95%, 99%): [1.73515654 2.2232977  2.68516938 3.70108823]


Statystyka KS: 0.15196141617255307
P-wartość: 7.525557912034457e-05
Statystyka U: 109249.0
P-wartość: 1.0163759592095675e-05
95% przedział ufności dla różnicy średnich według bootsrapu: (-12.892337479590958, 7.751386740568869)
P-wartość: 0.375
95% przedział ufności dla różnicy median według bootsrapu: (3.0, 7.0)
P-wartość: 0.0
